# Liar Dataset

Implement preprocessing functions in this notebook depending on whatever classifier you're training and just import whichever functions you need into your own model notebook.

## General Imports

In [1]:
from datasets import load_dataset

/Users/sawanpatel/Documents/UMich-Fall-2023/EECS595/Project/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Labels \\
0 ~ false \\
1 ~ half-true \\
2 ~ mostly true \\
3 ~ true \\
4 ~ barely true \\

## SVM Pre-processing

In [4]:
from sklearn import svm
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score, f1_score


In [2]:
svm_dataset = load_dataset('liar')

### Objective: Train 5 SVMs ('one-to-rest') to classify a Liar instance from 0-4

Split train/test datasets into 5 separate datasets, format labels in binary

In [9]:
train = svm_dataset['train']
test = svm_dataset['test']

In [15]:
def set_idx(example, idx):
    """Replaces all instance labels with binary labels given the intended label as input."""
    example['label'] = (example['label'] == idx)
    return example


labels = [0, 1, 2, 3, 4]
datasets = []
for i in labels:
    kwargs = {'idx': i}
    datasets.append(train.map(set_idx, fn_kwargs=kwargs))

Map: 100%|██████████| 10269/10269 [00:04<00:00, 2492.87 examples/s]
